# 🧠 Bayesian-AI Master Dashboard

**Objective:** Consolidated interface for System Verification, Debugging, and Learning Simulations.
**Workflow:** Preflight -> Component Check -> Data Verification -> Simulation -> Analysis

## 1. Preflight & Environment Checks ✈️
Verify Operational Mode, Environment, and CUDA Availability.

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output
from numba import cuda

# Add root to path
current_dir = Path(os.getcwd())
project_root = current_dir
while not (project_root / 'requirements.txt').exists():
    parent = project_root.parent
    if parent == project_root:
        break
    project_root = parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
print(f"Project Root: {project_root}")

from config.settings import OPERATIONAL_MODE, RAW_DATA_PATH
from training.orchestrator import get_data_source, load_data_from_directory

# 1. Check Operational Mode
print(f"\nOperational Mode: {OPERATIONAL_MODE}")
if OPERATIONAL_MODE != "LEARNING":
    print("🔴 WARNING: System should be in 'LEARNING' mode for this dashboard.")
else:
    print("🟢 Mode Checked")

# 2. Check CUDA (Strict)
try:
    if cuda.is_available():
        print(f"🟢 CUDA Available: {cuda.detect()}")
        device = cuda.get_current_device()
        print(f"   Device: {device.name}")
    else:
        print("🔴 CUDA Not Available. System requires GPU.")
except Exception as e:
    print(f"🔴 CUDA Check Failed: {e}")

# 3. Run Deep Audit
import cuda_modules.hardened_verification as hv
print("\nRunning 3-Stage CUDA Audit (Handshake -> Injection -> Handoff)...")
try:
    # This validates the CPU->GPU injection path and pure GPU handoff
    success = hv.run_audit()
    if success:
        print("🟢 Audit Passed! (See CUDA_Debug.log)")
    else:
        print("🔴 Audit Failed! (See CUDA_Debug.log)")
except Exception as e:
    print(f"🔴 Execution Error: {e}")

Project Root: c:\Users\reyse\OneDrive\Desktop\Bayesian-AI


ModuleNotFoundError: No module named 'scipy'

## 2. Core Component Verification ⚙️
Verify initialization of critical modules with **Strict GPU Enforcement**.

In [ ]:
from core.state_vector import StateVector
from core.bayesian_brain import BayesianBrain
from core.layer_engine import LayerEngine
from cuda_modules.velocity_gate import CUDAVelocityGate
from cuda_modules.pattern_detector import CUDAPatternDetector

print("--- Component Status ---")

# 1. StateVector
try:
    sv = StateVector.null_state()
    assert hash(sv) is not None
    print("🟢 StateVector: Operational")
except Exception as e:
    print(f"🔴 StateVector: Failed ({e})")

# 2. BayesianBrain
try:
    bb = BayesianBrain()
    print("🟢 BayesianBrain: Initialized")
except Exception as e:
    print(f"🔴 BayesianBrain: Failed ({e})")

# 3. LayerEngine (GPU ONLY)
try:
    if cuda.is_available():
        # Explicitly requesting GPU. If fallback logic exists in class, we want to ensure
        # we are using the GPU path.
        le = LayerEngine(use_gpu=True)
        if le.use_gpu:
             print("🟢 LayerEngine: Initialized (GPU Active)")
        else:
             print("🔴 LayerEngine: Fallback to CPU detected (Unexpected)")
    else:
        print("🔴 LayerEngine: Skipped (No CUDA)")
except Exception as e:
    print(f"🔴 LayerEngine: Failed ({e})")

# 4. CUDA VelocityGate (GPU ONLY)
try:
    if cuda.is_available():
        vg = CUDAVelocityGate(use_gpu=True)
        print(f"🟢 VelocityGate: Initialized (GPU={vg.use_gpu})")
    else:
        print("🔴 VelocityGate: Skipped (No CUDA)")
except Exception as e:
    print(f"🔴 VelocityGate: Failed ({e})")

# 5. Pattern Detector (GPU ONLY)
try:
    if cuda.is_available():
        pd_cuda = CUDAPatternDetector(use_gpu=True)
        print(f"🟢 PatternDetector: Initialized (GPU={pd_cuda.use_gpu})")
    else:
         print("🔴 PatternDetector: Skipped (No CUDA)")
except Exception as e:
    print(f"🔴 PatternDetector: Failed ({e})")

## 3. Data Setup & Verification 📊
Ensure `DATA/RAW` contains valid data and visualize a sample.

In [ ]:
try:
    print(f"Loading data from {RAW_DATA_PATH}...")
    # Load full dataset into memory for slicing
    full_data = load_data_from_directory(str(project_root / RAW_DATA_PATH))
    
    print(f"🟢 Data Loaded! Rows: {len(full_data)}")
    
    # Ensure timestamp conversion
    if 'timestamp' in full_data.columns:
        full_data['datetime'] = pd.to_datetime(full_data['timestamp'], unit='s' if full_data['timestamp'].dtype == 'float64' else None)
    else:
        # Fallback if index is datetime
        full_data['datetime'] = full_data.index
    
    # Show date range
    min_date = full_data['datetime'].min()
    max_date = full_data['datetime'].max()
    print(f"Date Range: {min_date} to {max_date}")
    
    # Visualize Sample
    plot_df = full_data.head(1000)
    fig = go.Figure(data=[go.Candlestick(x=plot_df['datetime'],
                open=plot_df['price'], high=plot_df['price'],
                low=plot_df['price'], close=plot_df['price'])])
    fig.update_layout(title='Price Sample (First 1000 ticks)', xaxis_rangeslider_visible=False)
    fig.show()
    
except Exception as e:
    print(f"🔴 Data Load Failed: {e}")
    full_data = None

In [ ]:
# UI Helper: Simulation Monitor
class SimulationMonitor:
    def __init__(self, title="Simulation Progress", max_steps=100):
        # Widgets
        self.lbl_title = widgets.HTML(f"<b>{title}</b>")
        self.progress = widgets.IntProgress(value=0, min=0, max=max_steps, description='Progress:', bar_style='info')
        self.lbl_status = widgets.Label(value="Initializing...")
        self.log_area = widgets.Output(layout={'border': '1px solid #ccc', 'height': '200px', 'overflow_y': 'scroll'})
        
        # Layout Container (Pop-up style)
        self.container = widgets.VBox(
            [self.lbl_title, self.progress, self.lbl_status, self.log_area],
            layout=widgets.Layout(
                border='2px solid #007bff',
                padding='10px',
                margin='10px 0',
                background_color='#f8f9fa',
                width='100%'
            )
        )
        
    def show(self):
        display(self.container)
        
    def log(self, message):
        with self.log_area:
            print(message)
            
    def update_progress(self, value, status=None):
        self.progress.value = value
        if status:
            self.lbl_status.value = status
            
    def complete(self, success=True):
        self.progress.value = self.progress.max
        self.progress.bar_style = 'success' if success else 'danger'
        self.lbl_status.value = "Complete" if success else "Failed"

## 4. Quick Learn: 3 Discrete Day Simulation 🎲
Randomly select 3 days and run isolated learning simulations to verify logic and win rates.

In [ ]:
import random
from training.orchestrator import TrainingOrchestrator

def run_quick_learn(monitor, verbose=False):
    if full_data is None:
        monitor.log("🔴 No data loaded.")
        monitor.complete(False)
        return

    # Identify unique days
    monitor.log("Identifying unique days...")
    unique_days = full_data['datetime'].dt.date.unique()
    
    if len(unique_days) < 3:
        monitor.log(f"🟡 Not enough data for 3 discrete days. Found: {len(unique_days)}")
        selected_days = unique_days
    else:
        selected_days = sorted(random.sample(list(unique_days), 3))
    
    monitor.log(f"Selected Days: {selected_days}")
    
    results = []
    total_steps = len(selected_days)
    monitor.progress.max = total_steps
    
    for i, day in enumerate(selected_days):
        monitor.update_progress(i, f"Simulating {day}...")
        monitor.log(f"--- Simulating {day} ---")
        
        day_mask = full_data['datetime'].dt.date == day
        day_data = full_data[day_mask].copy()
        
        # Initialize Orchestrator for this chunk
        debug_path = f'debug_outputs/quick_learn/debug_{day}.log' if verbose else None
        if verbose:
             monitor.log(f"📝 Logging to {debug_path}")

        orch = TrainingOrchestrator(
            asset_ticker='MNQ', 
            data=day_data, 
            output_dir='debug_outputs/quick_learn',
            verbose=verbose,
            debug_file=debug_path
        )
        
        # Run 1 iteration
        metrics = orch.run_training(iterations=1, params={'mode': 'quick_learn'})
        metrics['date'] = day
        results.append(metrics)
        
        monitor.log(f"Day Result: PnL=${metrics['pnl']:.2f}, WR={metrics['win_rate']:.1%}")
        
    monitor.complete(True)
    
    # Summary
    print("\n=== Quick Learn Summary ===")
    res_df = pd.DataFrame(results)
    display(res_df[['date', 'total_trades', 'win_rate', 'pnl']])

chk_verbose = widgets.Checkbox(value=False, description='Enable Verbose Logging')
btn_quick = widgets.Button(description="Run 3-Day Sim", button_style='info')

def on_quick_click(b):
    # Create and show monitor
    mon = SimulationMonitor("Quick Learn Simulation")
    mon.show()
    run_quick_learn(mon, verbose=chk_verbose.value)

btn_quick.on_click(on_quick_click)
display(widgets.HBox([btn_quick, chk_verbose]))

## 5. Full Learning Cycle 🚀
Execute the full training pipeline on all available data.

In [ ]:
import subprocess

def run_full_training(iterations=10, monitor=None):
    if monitor:
        monitor.log(f"Starting Full Training ({iterations} iterations)...")
    
    cmd = [
        "python", "training/orchestrator.py",
        "--data-dir", "DATA/RAW",
        "--iterations", str(iterations),
        "--output", "models/"
    ]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    # Approximate progress based on output lines or just spin
    monitor.progress.max = iterations
    current_iter = 0
    
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None:
            break
        if line:
            clean_line = line.strip()
            monitor.log(clean_line)
            if "Iter" in clean_line:
                current_iter += 1
                monitor.update_progress(current_iter, f"Running Iteration {current_iter}/{iterations}")
            
    if process.returncode == 0:
        monitor.complete(True)
        monitor.log("✅ Full Training Complete!")
    else:
        monitor.complete(False)
        monitor.log(f"❌ Training Failed (Code {process.returncode})")

btn_full = widgets.Button(description="Execute Full Learn", button_style='danger')

def on_full_click(b):
    mon = SimulationMonitor("Full Learning Cycle", max_steps=10)
    mon.show()
    run_full_training(iterations=10, monitor=mon)

btn_full.on_click(on_full_click)
display(btn_full)

## 6. Result Analysis & Profiling 📈
Analyze the probability tables and performance metrics.

In [ ]:
import pickle
import timeit
import itertools

def inspect_table(path, title):
    if not os.path.exists(path):
        print(f"⚪ {title}: File not found ({path})")
        return
    
    with open(path, 'rb') as f:
        data = pickle.load(f)
    table = data['table'] if isinstance(data, dict) and 'table' in data else data.table
    print(f"🟢 {title}: {len(table)} states learned.")
    
    # Convert to DataFrame for deep dive
    records = []
    for state, stats in table.items():
        total = stats['total']
        wins = stats['wins']
        if total > 0:
            records.append({
                'total': total,
                'wins': wins,
                'win_rate': wins/total,
                'L1': state.L1_bias,
                'L5': state.L5_trend
            })
    
    if records:
        df_stats = pd.DataFrame(records)
        # Win Rate vs Sample Size
        fig = px.scatter(df_stats, x='total', y='win_rate', title=f'{title}: Win Rate vs Sample Size',
                          hover_data=['L1', 'L5'])
        fig.show()

print("--- Model Status ---")
inspect_table('debug_outputs/quick_learn/probability_table.pkl', "Quick Learn Temp")
inspect_table('models/probability_table.pkl', "Main Model")

print("\n--- Performance Profiling ---")
print("Benchmarking StateVector Hashing...")
setup_code = "from core.state_vector import StateVector; sv = StateVector.null_state()"
t = timeit.timeit("hash(sv)", setup=setup_code, number=100000)
print(f"Create & Hash 100k states: {t:.4f}s")

print("\n--- DOE Simulation Preview ---")
param_grid = {
    'min_prob': [0.70, 0.75, 0.80, 0.85],
    'min_conf': [0.20, 0.30, 0.40],
    'stop_loss': [10, 20, 30],
    'kill_zones': ['tight', 'wide']
}
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(f"Total Parameter Combinations: {len(combinations)}")

## 7. Utilities 🛠️

In [ ]:
import shutil

def clean_pycache():
    print("Cleaning __pycache__...")
    count = 0
    for root, dirs, files in os.walk('.'):
        for d in dirs:
            if d == '__pycache__':
                shutil.rmtree(os.path.join(root, d))
                count += 1
    print(f"Removed {count} directories.")

btn_clean = widgets.Button(description="Clear PyCache")
btn_clean.on_click(lambda b: clean_pycache())
display(btn_clean)